In [222]:
import pandas as pd
import numpy as np
# from pivottablejs import pivot_ui
# import qgrid

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
pd.set_option("mode.chained_assignment", None)
import plotly.express as px
import tkinter as tk
from tkinter import filedialog
from datetime import datetime
import pyodbc
import time
import timeit
import sqlalchemy
import dateutil.relativedelta
import warnings
warnings.filterwarnings("ignore")
from PIL import Image, ImageTk

import os

## SQL數據處理class
- 需要寫兩個.py檔，1個是初始化數據庫使用，1個是用在日常更新使用，更新頻率為半小時

In [ ]:
# class for initialization
class SQL_data_init:
    
    last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
    
    def __init__(self, init=True) -> None:
        self._server = "10.226.0.53"
        self._database = "master"
        self._username = "BIUser"
        self._password = "test"
        self._cnxn = self.conn_server()
        self._server_ex = "10.226.61.50"
        self._database_ex = "SAP_DW"
        self._cnxn_ex = self.conn_server(fetch=False)
        self.init = init
        self.df = self.fetch_new_data(init=init)
        self.clean_df = self.data_preprocessing()
        self.loading2view()
        
            
    
    def conn_server(self, fetch=True):
        if fetch == True:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server,database=self._database ,uid=self._username,password=self._password)
        else:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server_ex,database=self._database_ex ,uid=self._username,password=self._password)
        return cnxn
    
    
    def fetch_new_data(self, init=True):
        if init == True:
            '''When init=False, initialize the data'''
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230101'"
        else:
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(self.last_month)
        df = pd.read_sql(script, self._cnxn)
        return df
    
    
    def data_preprocessing(self):
        df = self.df
        df["CreatedDateTime"] = datetime.now()
        df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']].fillna(0)
        return df
        
    
    
    def loading2view(self):
        df = self.clean_df
        conn = self._cnxn_ex
        cursor = conn.cursor()
        if self.init == True:
            cursor.execute("DELETE from ZMAW")
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        else:
            delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(self.last_month)
            cursor.execute(delete)
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        conn.commit()
        cursor.close()
        
        
if __name__ == "__main__":
    SQL_data_init()
        

In [ ]:
# Routine update
class SQL_data:
    
    last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
    
    def __init__(self, init=False) -> None:
        self._server = "10.226.0.53"
        self._database = "master"
        self._username = "BIUser"
        self._password = "test"
        self._cnxn = self.conn_server()
        self._server_ex = "10.226.61.50"
        self._database_ex = "SAP_DW"
        self._cnxn_ex = self.conn_server(fetch=False)
        self.init = init
        self.df = self.fetch_new_data(init=init)
        self.clean_df = self.data_preprocessing()
        self.loading2view()
        
            
    
    def conn_server(self, fetch=True):
        if fetch == True:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server,database=self._database ,uid=self._username,password=self._password)
        else:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server_ex,database=self._database_ex ,uid=self._username,password=self._password)
        return cnxn
    
    
    def fetch_new_data(self, init=True):
        if init == True:
            '''When init=False, initialize the data'''
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230201'"
        else:
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(self.last_month)
        df = pd.read_sql(script, self._cnxn)
        return df
    
    
    def data_preprocessing(self):
        df = self.df
        df["CreatedDateTime"] = datetime.now()
        df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']].fillna(0)
        return df
        
    
    
    def loading2view(self):
        df = self.clean_df
        conn = self._cnxn_ex
        cursor = conn.cursor()
        if self.init == True:
            cursor.execute("DELETE from ZMAW")
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        else:
            delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(self.last_month)
            cursor.execute(delete)
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        conn.commit()
        cursor.close()
        
        
if __name__ == "__main__":
    SQL_data()
        

In [ ]:
# Verify the result
server = "10.226.61.50"
database = "SAP_DW"
username = "BIUser"
password = "test"

conn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
cursor = conn.cursor()
df = pd.read_sql("select * from [dbo].[ZMAW]", con=conn)
df

In [ ]:
# 計算業績販社只有ZSO1和ZKE的訂單類型有效，本社是ZOR1（TBR(G)）
result = df[(df["中計"].str.contains("TBR|LSR2")) & (df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["配銷通路"].isin(["OT", "FT", "AG"]))]
result["銷售數量"].sum()
result
result["買方"].unique()



## SQL Server 連線

In [217]:
server = "10.226.0.53"
database = "master"
username = "BIUser"
password = "test"

In [218]:
cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)

In [111]:
cursor = cnxn.cursor()

In [ ]:
last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
last_month

In [ ]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(last_month)

In [219]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230201'"

In [ ]:
inventory_script = "select * from [10.212.27.198].FC0.dbo.V_BI_INVENTORY"

In [220]:
df = pd.read_sql(script, cnxn)

In [221]:
df = df[(df["中計"].str.contains("TBR|LSR2")) & (df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["配銷通路"].isin(["OT", "FT", "AG"]))]
df["交貨數量"].sum()

12054.0

In [ ]:
df[df["配銷通路"].isna()]

In [ ]:
inventory_status = df[df["料號"].str.contains("TBR|LSR")]

In [ ]:
inventory_status[inventory_status["未限制庫存"] != 0]["倉別名稱"].value_counts()

In [ ]:
# 查看系統所有訂單的script
current_mon = datetime.now().strftime("'%Y-%m'")
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = {}".format(current_mon)

In [ ]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where convert(varchar(7), 訂單建立日, 120) >= '2023-01'"

In [ ]:
## 所有訂單（包含販社和本社）
category_table = pd.read_excel("./ReferenceTable/20220801_category_types.xlsx")
cuscode_type = dict(zip(category_table["客戶代號"], category_table["大通路"]))
df["大通路"] = df["買方"].map(cuscode_type)

In [ ]:
cursor.close()

In [ ]:
df.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\sql_sever_data_from_2019ytd.pkl")

In [ ]:
df = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\sql_sever_data_from_2019ytd.pkl")

## 測試所有訂單的想法

In [2]:
server = "10.226.0.53"
database = "master"
username = "BIUser"
password = "test"

cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = '2023-02'"

In [22]:
df = pd.read_sql(script, cnxn)

In [23]:
df["大通路"] = df["彙總客戶"].map(cuscode_type)

In [33]:
df = df[(df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["大通路"].notna()) & (df["大通路"] == "BSSC") & (df["物料說明"].str.contains("315/80R225")) & (df["花紋別"].str.contains("R150|M729"))]

In [37]:
df.groupby(["買方全名", "業務員全名", "物料說明", "花紋別"])[["訂單數量"]].sum().reset_index().to_excel(r"C:\Users\kc.hsu\Desktop\2月特別販促清單.xlsx", index=False)

In [45]:
df.groupby(["大通路"])[["訂單數量"]].sum()

,訂單數量
大通路,
AG,413.0
BSSC,4870.0
BSSC-FLT,993.0
FLT-REP,643.0
OE,168.0


In [ ]:
logo = Image.open(r"D:\kc.hsu\My Documents\My Pictures\BS方形logo.png")

In [ ]:
logo.save(r"D:\kc.hsu\My Documents\My Pictures\BS方形logo.ico", format='ICO')

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\2021-2022_Size Data_All_TBR_rawling_data.xlsx", sheet_name="Sheet1")

In [ ]:
ob = df[(df["預算/前實績"] == "23OB_V2") & (df["國內市場財別"] == "TBR(G)")]

In [ ]:
ob_target = int(ob[(ob["年月"] == int(datetime.now().strftime("%Y%m"))) & (ob["通路明細"] == "FLT-REP")]["條數"].sum())

In [ ]:
class RealTimeAct(tk.Tk):
    server = "10.226.0.53"
    database = "master"
    username = "BIUser"
    password = "test"

    cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
    script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = '2023-01'"
    
    def __init__(self):
        super().__init__()
        self.wm_attributes("-topmost", 1)
        self.wm_title("每日TBR(G)販社訂單即時數據")
        # self.wm_maxsize(400, 150)
        self.connection = pd.read_sql(self.script, self.cnxn)
        # self.parent = parent
        
        self.refresh_data()
        # self.run()
        self.mainloop()
        
    def refresh_data(self):
        df = self.connection
        df["中計"] = df["中計"].fillna("")
        tbrg = df[df["中計"].str.contains("LSR2|TBR")]
        result = tbrg[tbrg["訂單類型"].isin(["ZKE", "ZSO1"])][["訂單數量", "交貨數量"]].sum()
        orderLabel = tk.Label(self, text="目前販社訂單總數: {}".format(int(result[0])), font=("Arial", 20)).grid(row=0, column=0)
        shipLabel = tk.Label(self, text="目前販社出貨總數: {}".format(int(result[1])), font=("Arial", 20)).grid(row=1, column=0)
        timeLabel = tk.Label(self, text="更新時間：{}".format(time.strftime("%Y-%m-%d %H:%M")), font=("Arial", 12)).grid(row=2, column=0)
        self.after(20000, self.refresh_data)
        # return result
    
    
    def run(self):
        # orderLabel = (self, text="目前販社訂單總數: {}".format(int(self.refresh_data[0])), font=("Arial", 20)).grid(row=0, column=0)
        # shipLabel = self.Label(self, text="目前販社出貨總數: {}".format(int(self.refresh_data[1])), font=("Arial", 20)).grid(row=1, column=0)
        # timeLabel = self.Label(self, text="更新時間：{}".format(time.strftime("%Y-%m-%d %H:%M")), font=("Arial", 12)).grid(row=2, column=0)
        pass
        
    

In [ ]:
RealTimeAct()

In [ ]:
while time.strftime("%H:%M", time.localtime()) != "17:00":
    df = pd.read_sql(script, cnxn)
    df["中計"] = df["中計"].fillna("")
    tbrg = df[df["中計"].str.contains("LSR2|TBR")]
    result = tbrg[tbrg["訂單類型"].isin(["ZKE", "ZSO1"])][["訂單數量", "交貨數量"]].sum()
    print(result)
    if result[1] + 1021 >= 9052:
        print("恭喜達標")
        break
    else:
        continue

## 使用BPCOM的數據來帶入Product name

In [ ]:
bpcom = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\產品規格相關\BPCOM\BPCOM_202208040543.xlsx")

In [ ]:
bpcom["Product Name"].map(lambda x: x.split(" ")[0]).value_counts()

In [ ]:
check_dict = dict(zip(bpcom["PrCode"], bpcom["BP"]))

In [ ]:
df["花紋別"] = df["物料"].map(check_dict)

In [ ]:
result = df[df["中計"].str.contains("TBR|LSR2")]

## 測試寫入SQL Server

In [30]:
server = "10.226.61.50"
database = "SAP_DW"
username = "BIUser"
password = "test"

In [31]:
conn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)

In [32]:
cursor = conn.cursor()

In [ ]:
delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(last_month)

In [ ]:
delete = "DELETE from ZMAW where 實際出貨日 >= '20221226'"

In [33]:
cursor.execute("DELETE from zmaw")

In [34]:
cursor.commit()

In [35]:
df = pd.read_sql("select * from [dbo].[ZMAW]", con=conn)
df

,組織,出貨點,所名,區域,訂單建立日,訂單文件日,訂單建立時,交貨文件日,計劃出貨日,實際出貨日,交單建立時,請款建立日,請款日期,請款建立時,訂單建立人,交單建立人,請款建立人,訂單備註,產品階層,訂單類型,訂單類型說明,交貨類型,交貨類型說明,請款類型,請款類型說明,訂單單號,訂單項次,交貨單號,交貨項次,請款文件,請款文件項次,發票號碼,買方,送貨方,彙總客戶,業務員,小胎業務員,大胎業務員,買方全名,買方簡稱,送貨方全名,送貨方簡稱,彙總客戶全名,彙總客戶簡稱,業務員全名,業務員簡稱,小胎業務員簡稱,大胎業務員簡稱,配銷通路,販社通路,大胎契約,小胎契約,付款條件,物料群組,分類,物料,物料說明,花紋別,Premium,吋別,中計,訂單數量,交貨數量,發票單價,發票未稅,稅額,發票合計,銷售數量,銷貨單價,未稅銷貨,CreatedDateTime


In [ ]:
df["CreatedDateTime"] = datetime.now()

In [ ]:
df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
       '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
       '未稅銷貨']].fillna(0)

In [ ]:
df.columns

In [ ]:
for index, row in df.iterrows():
    cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員, 大胎業務員, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
    # print(tuple(row))

In [ ]:
conn.commit()

In [ ]:
cursor.close()

## 數據轉換function

In [223]:
cur_month = datetime.today().date().strftime("%Y-%m") + "-01"

In [224]:
class DataExtract:
    
    type_keys = ['訂單日期', '實際出貨日', '訂單單號', '訂單項次', '買方', '物料', '物料說明', '國別', '銷售數量', '銷貨單價']
    type_values = [str, str, str, str, str, str, str, str, int, int]
    type_dict = dict(zip(type_keys, type_values))
    
    def __init__(self):
        self.path = self.get_path()
        
    def get_path(self):
        paths = []
        while True:
            answer = input("選擇路徑?(Y/N)").upper()
            if answer == "Y":
                root = tk.Tk()
                root.withdraw()
                file_path = filedialog.askopenfilenames()
                paths += file_path    
            else:
                break
        return paths
            
        
    def read_data(self):
        dfs = []
        # path = self.path.replace("\\", "/")
        for path in self.path:
            if path.split(".")[-1] == "csv" and "ag" in path:
                ag = pd.read_csv(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            elif path.split(".")[-1] == "pkl" and "ag" in path:
                ag = pd.read_pickle(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            elif path.split(".")[-1] == "csv" and "sc" in path:
                sc = pd.read_csv(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
            elif path.split(".")[-1] == "pkl" and "sc" in path:
                sc = pd.read_pickle(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
            elif path.split(".")[-1] == "xlsx" and "ag" in path:
                ag = pd.read_excel(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            else:
                sc = pd.read_excel(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
        data = pd.concat(dfs)
        data = data.drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last")
        return data

    def ag_extract(self, df):
        # ag = self.read_data()
        ag = df
        ag.columns = ag.columns.str.strip()
        ag = ag[(ag["S_配銷通路"].isin(["AG", "FT"])) & (ag["S_中計商品代號1"].isin(["LR2", "TR2", "TR3", "TR5", "TR4"]))]
        ag = ag[["D_文件日期", "D_實際發貨日期", "S_訂單單號", "S_項次", "S_買方(客戶號碼)", "S_物料編號", "S_物料說明", "S_國別", "S_數量", "S_項目淨值"]]
        ag.columns = self.type_keys
        # ag = ag.astype(self.type_dict)
        return ag
    
    def sc_extract(self, df):
        # sc = self.read_data()
        sc = df
        sc.columns = sc.columns.str.strip()
        sc = sc[sc["中計"].isin(["LSR2", "TBR3", "TBR5", "TBR2", "TBR4"])]
        sc = sc[["訂單建立日", "實際出貨日", "訂單單號", "訂單項次", "買方", "物料", "物料說明", "國別", "銷售數量", "銷貨單價"]]
        sc.columns = self.type_keys
        # sc = sc.astype(self.type_dict)
        return sc
    
    def sales_preprocessing(self):
        df = self.read_data()
        df["訂單日期"] = df["訂單日期"].dt.date
        df["實際出貨日"] = df["實際出貨日"].dt.date
        df = df.astype(self.type_dict)
        df["訂單單號"] = df["訂單單號"].map(lambda x: x.replace(".0", "") if "." in x else x)
        df["訂單項次"] = df["訂單項次"].map(lambda x: x.replace(".0", "") if "." in x else x)
        df["銷貨單價"] = df["銷貨單價"].fillna(0).astype(int)
        return df

In [241]:
result = DataExtract().sales_preprocessing()

### 開啟客戶對照表

In [242]:
# 開啟客編客戶對照表
# category_table = pd.read_csv("./ReferenceTable/category_types.csv")
category_table = pd.read_excel("./ReferenceTable/20220801_category_types.xlsx")
# cat_table = qgrid.show_grid(category_table, show_toolbar=True)
# cat_table

# 如果有修改的話，修改完可以用.get_changed_df()來獲得修改過後的表格

In [243]:
cuscode_cus = dict(zip(category_table["客戶代號"], category_table["合併客戶名稱"]))
cuscode_type = dict(zip(category_table["客戶代號"], category_table["大通路"]))
cuscode_channel = dict(zip(category_table["客戶代號"], category_table["小通路"]))
sales_dict = dict(zip(category_table["客戶代號"], category_table["業務名稱"]))
sales_area = dict(zip(category_table["客戶代號"], category_table["負責區域"]))
sales_branch = dict(zip(category_table["客戶代號"], category_table["所屬區域"]))
cus_address = dict(zip(category_table["客戶代號"], category_table["住址"]))
cus_city = dict(zip(category_table["客戶代號"], category_table["城市"]))
rep_dict = dict(zip(category_table["客戶代號"], category_table["母通路"]))
liwei_dict = dict(zip(category_table["客戶代號"], category_table["liwei"]))
ranking = dict(zip(category_table["客戶代號"], category_table["Ranking"]))
cus_area = dict(zip(category_table["客戶代號"], category_table["客戶所在地分區"]))

In [244]:
pcode = pd.read_excel("./ReferenceTable/20220721_PCODE整理表.xlsx")
size_dict = dict(zip(pcode["Old PCODEs"], pcode["size"]))
pattern_dict = dict(zip(pcode["Old PCODEs"], pcode["pattern"]))
origin_dict = dict(zip(pcode["Old PCODEs"], pcode["p_origin"]))
tbr_prem = dict(zip(pcode["Old PCODEs"], pcode["TBR Premium"]))
group = dict(zip(pcode["Old PCODEs"], pcode["group"]))

In [245]:
result.reset_index(drop=True).to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\rolling_sales_data.pkl")
long = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\rolling_sales_data.pkl")

In [246]:
long["客戶名稱"] = long["買方"].map(cuscode_cus)
long["母通路"] = long["買方"].map(rep_dict)
long["大通路"] = long["買方"].map(cuscode_type)
long["小通路"] = long["買方"].map(cuscode_channel)
long["日報通路"] = long["買方"].map(liwei_dict)
long["業務名稱"] = long["買方"].map(sales_dict)
long["業務負責區域"] = long["買方"].map(sales_area)
long["業務所屬區域"] = long["買方"].map(sales_branch)
long["客戶登記地址"] = long["買方"].map(cus_address)
long["客戶城市"] = long["買方"].map(cus_city)
long["客戶等級"] = long["買方"].map(ranking)
long["規格"] = long["物料"].map(size_dict)
long["花紋"] = long["物料"].map(pattern_dict)
long["工廠"] = long["物料"].map(origin_dict)
long["TBR Premium"] = long["物料"].map(tbr_prem)
long["輪胎類別"] = long["物料"].map(group)
long["品牌"] = long["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")
long["客戶所在地區分區"] = long["買方"].map(cus_area)

In [247]:
long[long["客戶名稱"].isna()]["買方"].unique()

array([], dtype=object)

In [248]:
long["銷售數量"].sum()

11996

In [249]:
long[long["大通路"] != "OE"]["銷售數量"].sum()

11996

In [250]:
# historical = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")
historical = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")

In [251]:
historical["客戶名稱"] = historical["買方"].map(cuscode_cus)
historical["業務名稱"] = historical["買方"].map(sales_dict)
historical["客戶等級"] = historical["買方"].map(ranking)
historical["業務負責區域"] = historical["買方"].map(sales_area)
historical["業務所屬區域"] = historical["買方"].map(sales_branch)
historical["客戶城市"] = historical["買方"].map(cus_city)
historical["母通路"] = historical["買方"].map(rep_dict)
historical["大通路"] = historical["買方"].map(cuscode_type)
historical["小通路"] = historical["買方"].map(cuscode_channel)
historical["日報通路"] = historical["買方"].map(liwei_dict)
historical["客戶所在地區分區"] = historical["買方"].map(cus_area)

In [252]:
historical["訂單單號"] = historical["訂單單號"].map(lambda x: x.replace(".0", "") if "." in x else x)
historical["訂單項次"] = historical["訂單項次"].map(lambda x: x.replace(".0", "") if "." in x else x)

In [253]:
historical = historical.drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last")

In [254]:
historical = historical[historical["實際出貨日"] < cur_month]

In [255]:
# historical.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")
# historical.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.xlsx", index=False)

In [256]:
pd.concat([historical, long]).drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last").to_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet", index=False)
pd.concat([historical, long]).drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last").to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.xlsx", index=False)

## 每日庫存整理

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory_status.xlsx")

In [ ]:
df["類別"] = df["物料"].map(group)

In [ ]:
df = df[~df["類別"].isna()]

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df["花紋"] = df["物料"].map(pattern_dict)
df["規格"] = df["物料"].map(size_dict)
df["工廠"] = df["物料"].map(origin_dict)
df["TBR Premium"] = df["物料"].map(tbr_prem)
df["品牌"] = df["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")
df["收貨日期"] = df["收貨日期"].dt.date

In [ ]:
df = df[["物料", "類別", "花紋", "規格", "工廠", "TBR Premium", "品牌", "可用的庫存", "收貨日期", "總庫存", "入庫庫存", "撿貨數量"]]

In [ ]:
df["撿貨數量"] = df["撿貨數量"] * -1

In [ ]:
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory_status_處理過.xlsx", index=False)

### 將xlsx批次轉csv或pkl檔

In [ ]:
"ag" in paths[1]

In [ ]:
paths = []
while True:
    answer = input("選擇路徑?(Y/N)").upper()
    if answer == "Y":
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilenames()
        paths += file_path    
    else:
        break

In [ ]:
for path in paths:
    pd.read_excel(path).to_csv(path.split(".")[0] + ".csv", index=False)

In [ ]:
for path in paths:
    pd.read_csv(path).to_pickle(path.split(".")[0] + ".pkl")

In [ ]:
sc_data = pd.concat([DataExtract(path).sc_extract() for path in paths])

In [ ]:
ag_data = pd.concat([DataExtract(path).ag_extract() for path in paths])

In [ ]:
ag_data["訂單日期"] = ag_data["訂單日期"].map(lambda x: x.split(" ")[0] if len(x) > 10 else x)

In [ ]:
sc_data["銷售數量"] = sc_data["銷售數量"].astype(int)

In [ ]:
result = pd.concat([sc_data, ag_data])

In [ ]:
result["訂單項次"] = result["訂單項次"].astype(str).map(lambda x: x.split(".")[0])

In [ ]:
result["銷貨單價"] = result["銷貨單價"].fillna(0).astype(int)

In [ ]:
result["訂單單號"] = result["訂單單號"].astype(str).map(lambda x: x.split(".")[0])

In [ ]:
result.reset_index(drop=True).to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\202201-08.pkl")

In [ ]:
long = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")

In [ ]:
long["客戶名稱"] = long["買方"].map(cuscode_cus)
long["母通路"] = long["買方"].map(rep_dict)
long["大通路"] = long["買方"].map(cuscode_type)
long["小通路"] = long["買方"].map(cuscode_channel)
long["日報通路"] = long["買方"].map(liwei_dict)
long["業務名稱"] = long["買方"].map(sales_dict)
long["業務負責區域"] = long["買方"].map(sales_area)
long["業務所屬區域"] = long["買方"].map(sales_branch)
long["客戶登記地址"] = long["買方"].map(cus_address)
long["客戶城市"] = long["買方"].map(cus_city)
long["客戶等級"] = long["買方"].map(ranking)

In [ ]:
long["規格"] = long["物料"].map(size_dict)
long["花紋"] = long["物料"].map(pattern_dict)
long["工廠"] = long["物料"].map(origin_dict)
long["TBR Premium"] = long["物料"].map(tbr_prem)
long["輪胎類別"] = long["物料"].map(group)
long["品牌"] = long["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")

In [ ]:
long

In [ ]:
long.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")
long.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.xlsx", index=False)

## 相關係數分析

In [ ]:
test = historical[['物料', '國別', '銷售數量', '銷貨單價', '客戶名稱', '小通路', '業務名稱', '業務負責區域', '業務所屬區域', '客戶城市', '規格', '花紋', '工廠', 'TBR Premium', '輪胎類別', '品牌',
       '客戶等級']]

In [ ]:
pd.get_dummies(test).corr(method="spearman")

## 小徐要的yahoo汽車對比普利司通輪胎規格花紋

In [ ]:
# 整理李濰的POS PSR規格花紋對照表
df = pd.read_clipboard()

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df["花紋"] = df["花紋"].str.strip()
df["規格"] = df["規格"].str.strip()
df["料號"] = df["料號"].str.strip()

In [ ]:
# 讀取各個廠牌的規格花紋
datas = []
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()

In [ ]:
for path in paths:
    if "conti" not in path:
        df = pd.read_excel(path)
        datas.append(df)
    else:
        conti = pd.read_excel(path)

In [ ]:
mi = pd.concat(datas)

In [ ]:
mi = mi[["規格", "花紋"]]

In [ ]:
all_brand = pd.concat([mi, conti])

In [ ]:
result = all_brand.groupby(["規格"])["花紋"].apply(list).reset_index()

In [ ]:
all_brand_list = dict(zip(result["規格"], result["花紋"]))

In [ ]:
yahoo["competitor"] = yahoo["前輪尺寸"].map(all_brand_list)

In [ ]:
yahoo["competitor"].fillna("", inplace=True)

In [ ]:
competitor_table = pd.DataFrame(yahoo["competitor"].to_list())

In [ ]:
competitor_table

In [ ]:
competitor_table.columns = ["Competitor 1", "Competitor 2", "Competitor 3", "Competitor 4", "Competitor 5", "Competitor 6", "Competitor 7", "Competitor 8", "Competitor 9", 
                            "Competitor 10", "Competitor 11", "Competitor 12", "Competitor 13", "Competitor 14", "Competitor 15", "Competitor 16", "Competitor 17", "Competitor 18", "Competitor 19"]

In [ ]:
export_data = pd.concat([yahoo, competitor_table], axis=1)

In [ ]:
export_data.to_excel(r"C:\Users\kc.hsu\Desktop\20221215_yahoo汽車對比競品花紋大全.xlsx", index=False)

In [ ]:
# 使用groupby將所有一樣規格的花紋打包到list
result = df.groupby(["規格"])["料號"].apply(list).reset_index()

In [ ]:
size_table = dict(zip(result["規格"], result["料號"]))

In [ ]:
# 讀取yahoo汽車規格
yahoo = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\Yahoo汽車資訊\20221212_yahoo汽車對比普利司通花紋.xlsx")

In [ ]:
yahoo = pd.read_clipboard()

In [ ]:
yahoo["BS規格"] = yahoo["前輪尺寸"].map(size_table)

In [ ]:
yahoo["BS規格"].fillna("", inplace=True)

In [ ]:
final = pd.concat([yahoo, pd.DataFrame(yahoo["BS規格"].to_list())], axis=1)

In [ ]:
final.columns = [  '年份',   '品牌',   '車系',   '車型',   '價格', '動力型式',  '排氣量', '驅動型式', '車身型式',
         '車重', '前輪尺寸', '後輪尺寸', 'BS規格',      "BPC 1",      "BPC 2",      "BPC 3",      "BPC 4",      "BPC 5",
            "BPC 6",      "BPC 7",      "BPC 8"]

In [ ]:
pattern_dict = dict(zip(df["料號"], df["花紋"]))

In [ ]:
for i, j in enumerate(["BPC 1",      "BPC 2",      "BPC 3",      "BPC 4",      "BPC 5",
            "BPC 6",      "BPC 7",      "BPC 8"], start=1):
    final[i] = final[j].map(pattern_dict)

In [ ]:
final.columns = [   '年份',    '品牌',    '車系',    '車型',    '價格',  '動力型式',   '排氣量',  '驅動型式',
        '車身型式',    '車重',  '前輪尺寸',  '後輪尺寸',  'BS規格', 'BPC 1', 'BPC 2', 'BPC 3',
       'BPC 4', 'BPC 5', 'BPC 6', 'BPC 7', 'BPC 8',       'Pattern 1',       'Pattern 2',       'Pattern 3',
             'Pattern 4',       'Pattern 5',       'Pattern 6',       'Pattern 7',       'Pattern 8']

In [ ]:
export_data = final[['年份',    '品牌',    '車系',    '車型',    '價格',  '動力型式',   '排氣量',  '驅動型式',
        '車身型式',    '車重',  '前輪尺寸',  '後輪尺寸', 'BPC 1', 'Pattern 1', 'BPC 2', 'Pattern 2', 'BPC 3', 'Pattern 3',
       'BPC 4', 'Pattern 4', 'BPC 5', 'Pattern 5', 'BPC 6', 'Pattern 6', 'BPC 7', 'Pattern 7', 'BPC 8', 'Pattern 8']]

In [ ]:
export_data.to_excel(r"C:\Users\kc.hsu\Desktop\20221212_yahoo汽車對比普利司通花紋.xlsx", index=False)

## 歷史記錄

In [ ]:
df = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")

In [ ]:
result = df[df["花紋"] == "R154"]

In [ ]:
result = result[["實際出貨日", "買方", "物料", "規格", "花紋", "客戶名稱", "銷售數量"]]

In [ ]:
result.to_excel(r"C:\Users\kc.hsu\Desktop\20230201_R154銷售.xlsx", index=False)

## 產品促銷試算

In [ ]:
df = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")

In [ ]:
## AG 8% discount from the list price without tax
## FLT(BSSC and REP all shows the real net with tax)
## read BSSC price with S net
s_price = pd.read_clipboard()

In [ ]:
s_price["A net"] = s_price["A net"].str.replace(",", "").astype(int)

In [ ]:
df = df[(df["實際出貨日"] >= "2023-02-01") & (df["大通路"] != "OE")]

In [ ]:
s_price["描述"] = s_price["規格"] + " " + s_price["花紋"] + " " + s_price["國別"]

In [ ]:
price_dict = dict(zip(s_price["描述"], s_price["A net"]))

In [ ]:
df["描述"] = df["規格"] + " " + df["花紋"] + " " + df["國別"]

In [ ]:
df["S net"] = df["描述"].map(price_dict)

In [ ]:
df["Price"] = df.apply(lambda x: (x["銷貨單價"] / 1.05) * 0.92 if x["大通路"] == "AG" else x["銷貨單價"] / 1.05 if x["大通路"] in ["BSSC-FLT", "FLT-REP"] else x["S net"], axis=1 )

In [ ]:
df["總NET"] = df["銷售數量"] * df["Price"]

In [ ]:
df.groupby(["大通路"])[["銷售數量", "總NET"]].sum().reset_index().to_clipboard()